In dit document worden de onderstaande instellingen/functies gebruikt:

In [79]:
# Remove the scientific notation for 'big' numbers.
options(scipen=999)

In [80]:
# Add the column and row numbers to the matrix for easier indexing.
# It preserves the original naming.
colrownames <- function(M) {
    cn=colnames(M)
    cn=paste(cn, ' (', c(1:length(M[1,])), ')', sep='')
    colnames(M)=cn
    rn=rownames(M)
    rn=paste(rn, ' (', c(1:length(M[,1])), ')', sep='')
    rownames(M)=rn
    M
}

# LP probleem

Het LP-probleem voor opgave 34 is als volgt:

$$ \begin{aligned}
\min 3000x+4500y&+4000z\\
x+y+z &= 100 \qquad&\text{(1)} \\
x &\geq 30 \qquad&\text{(2)} \\
-x+y &\leq 0 \qquad&\text{(3)} \\
-y+2z &\leq 0 \qquad&\text{(4)}\\
z &\leq 10 \qquad&\text{(5)}
\end{aligned}$$


Alle mogelijke combinaties van $\leq, \geq, =$ komen hierin voor. Ook moet het probleem worden omgezet in een maximalisatieprobleem. Doordat er kunstmatige variabelen nodig zijn, wordt er een twee-fasen model gebruikt.

# Canonieke vorm

Eerst stellen we de doelfunctie op: $d=3000x+4500y+4000z$. Omdat dit een minimalisatie probleem is, en we er een maximalisatieprobleem van willen maken, wordt $d^*=-d$. De doelfunctie wordt dan: $-d=-3000x-4500y-4000z \iff d^*=-3000x-4500y-4000z$.

Vervolgens herschrijven we de andere restricties in de canonieke vorm, dit geeft de volgende vergelijkingen:

$$ \tag{0} d^*+3000x+4500y+4000z = 0 $$
$$ \tag{1} x+y+z+A_1=100 $$
$$ \tag{2} x-S_2+A_2=30 $$
$$ \tag{3} -x+y+s_3=0 $$
$$ \tag{4} -y+2z+s_4=0 $$
$$ \tag{5} z+s_5=10 $$

Omdat er kunstmatige variabelen worden gebruikt hebben we een extra doelfunctie nodig, namelijk: $A=-A_1-A_2$. Deze doelfunctie wordt gemaximaliseerd tot $0$. In de canonieke vorm schrijven we dit als $A+A_1+A_2=0  \quad(0^*)$. Echter mogen $A_1$ en $A_2$ maar één keer in de vergelijkingen voorkomen omdat dit basisvariabelen zijn. Dit probleem is als volgt op te lossen: $(0^*)-(1)-(2)$. Dit geeft: $A+A_1+A_2-(x+y+z+A_1)-(x-S_2+A_2)=0-100-30$ ofwel, $A-2x-y-z+S_2=-130$.

Hieronder staan alle vergelijkingen in de canonieke vorm:

$$ \tag{0*} A-2x-y-z+S_2=-130 $$
$$ \tag{0} d^*+3000x+4500y+4000z = 0 $$
$$ \tag{1} x+y+z+A_1=100 $$
$$ \tag{2} x-S_2+A_2=30 $$
$$ \tag{3} -x+y+s_3=0 $$
$$ \tag{4} -y+2z+s_4=0 $$
$$ \tag{5} z+s_5=10 $$

Nu kunnen we dit omzetten naar een simplex tableau.

# Simplex

Eerst stellen we het begintableau op:

In [81]:
A=c(1,0,0,0,0,0,0)
d=c(0,1,0,0,0,0,0)
x=c(-2,3000,1,1,-1,0,0)
y=c(-1,4500,1,0,1,-1,0)
z=c(-1,4000,1,0,0,2,1)
A1=c(0,0,1,0,0,0,0)
S2=c(1,0,0,-1,0,0,0)
A2=c(0,0,0,1,0,0,0)
s3=c(0,0,0,0,1,0,0)
s4=c(0,0,0,0,0,1,0)
s5=c(0,0,0,0,0,0,1)
RHS=c(-130,0,100,30,0,0,10)
M=cbind(A,d,x,y,z,A1,S2,A2,s3,s4,s5,RHS)
rownames(M)=c('A','d*','A1','A2','s3','s4', 's5')
M=colrownames(M)

# Iteratie 1 (A)
M

,A (1),d (2),x (3),y (4),z (5),A1 (6),S2 (7),A2 (8),s3 (9),s4 (10),s5 (11),RHS (12)
A (1),1,0,-2,-1,-1,0,1,0,0,0,0,-130
d* (2),0,1,3000,4500,4000,0,0,0,0,0,0,0
A1 (3),0,0,1,1,1,1,0,0,0,0,0,100
A2 (4),0,0,1,0,0,0,-1,1,0,0,0,30
s3 (5),0,0,-1,1,0,0,0,0,1,0,0,0
s4 (6),0,0,0,-1,2,0,0,0,0,1,0,0
s5 (7),0,0,0,0,1,0,0,0,0,0,1,10


Wat opvalt in het begintableau, met uitzondering van vergelijking $(0^*)$, voor de toegevoegde variabelen is het volgende:

 1. Een spelingsvariabel (SlackN) $s_i$ is altijd $\geq$ 0 in het begintableau.
 2. Een surplusvariabel (Surp.N) $S_i$ is altijd $\leq$ 0 in het begintableau.
 3. Een kunstmatige variabel (Art.N) $A_i$ is altijd $\geq$ 0 in het begintableau.
 
Deze observatie kan worden gebruikt om het begintableau op correctheid te controleren, voordat er wordt begonnen met vegen.

De meeste negatieve waarde is in dit geval $x=-2$, vegen geeft:

In [82]:
# Iteratie 2 (A)
I2=M
I2[1,]=I2[1,]+2*I2[4,]
I2[2,]=I2[2,]-3000*I2[4,]
I2[3,]=I2[3,]-I2[4,]
I2[5,]=I2[5,]+I2[4,]
I2

,A (1),d (2),x (3),y (4),z (5),A1 (6),S2 (7),A2 (8),s3 (9),s4 (10),s5 (11),RHS (12)
A (1),1,0,0,-1,-1,0,-1,2,0,0,0,-70
d* (2),0,1,0,4500,4000,0,3000,-3000,0,0,0,-90000
A1 (3),0,0,0,1,1,1,1,-1,0,0,0,70
A2 (4),0,0,1,0,0,0,-1,1,0,0,0,30
s3 (5),0,0,0,1,0,0,-1,1,1,0,0,30
s4 (6),0,0,0,-1,2,0,0,0,0,1,0,0
s5 (7),0,0,0,0,1,0,0,0,0,0,1,10


De meeste negatieve waarde is in dit geval $y=-1$ en $z=-1$, beiden zijn gelijk, dus het wordt $y$ omdat dit de eerste is.

In [83]:
# Iteratie 3 (A)
I3=I2
I3[1,]=I3[1,]+I3[5,]
I3[2,]=I3[2,]-4500*I3[5,]
I3[3,]=I3[3,]-I3[5,]
I3[6,]=I3[6,]+I3[5,]
I3

,A (1),d (2),x (3),y (4),z (5),A1 (6),S2 (7),A2 (8),s3 (9),s4 (10),s5 (11),RHS (12)
A (1),1,0,0,0,-1,0,-2,3,1,0,0,-40
d* (2),0,1,0,0,4000,0,7500,-7500,-4500,0,0,-225000
A1 (3),0,0,0,0,1,1,2,-2,-1,0,0,40
A2 (4),0,0,1,0,0,0,-1,1,0,0,0,30
s3 (5),0,0,0,1,0,0,-1,1,1,0,0,30
s4 (6),0,0,0,0,2,0,-1,1,1,1,0,30
s5 (7),0,0,0,0,1,0,0,0,0,0,1,10


De meeste negatieve waarde is nu $S_2$, vegen geeft:

In [84]:
# Iteratie 4 (A)
I4=I3
I4[1,]=I4[1,]+I4[3,]
I4[3,]=I4[3,]
I4[2,]=I4[2,]-7500/2*I4[3,]
I4[4,]=I4[4,]+1/2*I4[3,]
I4[5,]=I4[5,]+1/2*I4[3,]
I4[6,]=I4[6,]+1/2*I4[3,]
I4[3,]=I4[3,]/2
I4

,A (1),d (2),x (3),y (4),z (5),A1 (6),S2 (7),A2 (8),s3 (9),s4 (10),s5 (11),RHS (12)
A (1),1,0,0,0,0.0,1.0,0,1,0.0,0,0,0
d* (2),0,1,0,0,250.0,-3750.0,0,0,-750.0,0,0,-375000
A1 (3),0,0,0,0,0.5,0.5,1,-1,-0.5,0,0,20
A2 (4),0,0,1,0,0.5,0.5,0,0,-0.5,0,0,50
s3 (5),0,0,0,1,0.5,0.5,0,0,0.5,0,0,50
s4 (6),0,0,0,0,2.5,0.5,0,0,0.5,1,0,50
s5 (7),0,0,0,0,1.0,0.0,0,0,0.0,0,1,10


Nu zijn we klaar met de eerste fase en is $A=0$. Vervolgens gaan we verder met het optimaliseren van de doelfunctie $d^*$. Let er op dat we nu alle kolommen met $A$ en de eerste rij negeren.

De meest negatieve waarde is $A_1$, maar deze kolom telt niet meer mee. De volgende waarde is $s_3$, vegen geeft:

In [85]:
# Iteratie 1 (d*)
I5=I4
I5[5,]=I5[5,]*2
I5[2,]=I5[2,]+I5[5,]*750
I5

,A (1),d (2),x (3),y (4),z (5),A1 (6),S2 (7),A2 (8),s3 (9),s4 (10),s5 (11),RHS (12)
A (1),1,0,0,0,0.0,1.0,0,1,0.0,0,0,0
d* (2),0,1,0,1500,1000.0,-3000.0,0,0,0.0,0,0,-300000
A1 (3),0,0,0,0,0.5,0.5,1,-1,-0.5,0,0,20
A2 (4),0,0,1,0,0.5,0.5,0,0,-0.5,0,0,50
s3 (5),0,0,0,2,1.0,1.0,0,0,1.0,0,0,100
s4 (6),0,0,0,0,2.5,0.5,0,0,0.5,1,0,50
s5 (7),0,0,0,0,1.0,0.0,0,0,0.0,0,1,10


Alle getallen in de rij $d^*$ zijn nu positief, we zijn dus klaar. De rechterkant is nu $-300.000$ en als we $d^*=-d$ toepassen (het was een minimalisatieprobleem), dan is de minimale waarde $300.000$.

# Oplossing

De oplossing is $(x,y,z)=(0,1500,1000)$ met een minimale waarde van $€300.000$.